# Some Typical Financial Calculations

# Load Packages

The Roots and Distributions packages have been used in the previous notebooks, but StatsBase not. It contains methods for estimating means, variances, ranks, correlations etc. See https://github.com/JuliaStats/StatsBase.jl. 

In [2]:
using StatsBase, Roots, Distributions
#using Dates, DelimitedFiles      #Julia 0.7

include("printmat.jl")            #function for prettier matrix printing

printmatDate

In [3]:
using Plots

backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
else    
    gr(size=(600,400))
end

Plots.GRBackend()

# CAPM and Return Autocorrelation

The CAPM regression is

$R_{it}^{e}  =\alpha_{i}+\beta_{i}R_{mt}^{e}+\varepsilon_{it}$,

where $R_{mt}^{e}$ is the market excess return. Theory says that $\alpha=0$, which is easily tested.

### Load Data

In [4]:
x   = readdlm("Data/MyData.csv",',',skipstart=1)        #monthly return data
ym  = round.(Int,x[:,1])     #yearmonth, like 200712
Rme = x[:,2]                 #market excess return
Rf  = x[:,3]                 #interest rate
R   = x[:,4]                 #return small growth stocks
Re  = R - Rf                 #excess returns
T   = size(Rme,1)

df = Dates.DateFormat("yyyymm")  #precompile the date format
dN = Date.(string.(ym),df)       #convert to string and then Julia Date
printmat(dN[1:4])

1979-01-01
1979-02-01
1979-03-01
1979-04-01



### CAPM regression

In [5]:
x    = [ones(T) Rme]             #regressors
y    = copy(Re)                  #to get standard OLS notation
b    = x\y                       #OLS
u    = y - x*b                   #residuals
covb = inv(x'x)*var(u)           #cov(b), see any textbook
stdb = sqrt.(diag(covb))         #std(b)
R2a  = 1 - var(u)/var(y)

println("     coeff      std     t-stat")
printmat([b stdb b./stdb])
printlnPs("R2: ",R2a)
printlnPs("no. of observations: ",T)

     coeff      std     t-stat
    -0.504     0.304    -1.656
     1.341     0.066    20.427

R2:      0.519
no. of observations:        388


## Return Autocorrelations

That is, the correlation of $R_{t}^{e}$ and $R_{t-s}^{e}$. 

(It is easily shown that the t-stat of an autocorrelation is $\sqrt{T}$ times the autocorrelation.)

In [6]:
plags = 1:5
xCorr = autocor(Re,plags)         #using the StatsBase package

println("Autocorrelations of the excess returns in Re")
println("\n     lag   autocorrr    t-stat")
printmat([plags xCorr sqrt(T)*xCorr])

Autocorrelations of the excess returns in Re

     lag   autocorrr    t-stat
     1.000     0.216     4.253
     2.000     0.002     0.046
     3.000    -0.018    -0.359
     4.000    -0.065    -1.289
     5.000    -0.027    -0.536



# A Trading Strategy

The next cell implements a very simple (momentum) trading strategy: if $R_{t-1}^{e}\ge0$, then we hold the market index and shorten the riskfree from $t-1$ to $t$. Instead,  if $R_{t-1}^{e}<0$, then we do the opposite.

This simple strategy could be coded without using a loop, but "vectorization" does not speed up much. 

In [7]:
w  = fill(NaN,T)
Rp = fill(NaN,T)
for t = 2:T
  w[t]  = (Re[t-1] < 0)*(-1) + (Re[t-1] >= 0)*1       #w is -1 or 1
  Rp[t] = w[t]*Re[t] 
end

printlnPs("The mean excess return of the strategy and a passive portfolio are: ",mean(Rp[2:end]),mean(Re[2:end]))

The mean excess return of the strategy and a passive portfolio are:      2.228     0.277


# Value at Risk

The next cell first constructs an simple estimate of $\sigma_t^2$ as a backward looking moving average (the RiskMetrics approach)

$\sigma_t^2 = \lambda \sigma_{t-1}^2 + (1-\lambda) (R_{t-1} -\mu)^2$,
where $\mu$ is the average return (for all data).

Then, we calculate the 95% VaR by assuming a $N(\mu,\sigma_t^2)$ distribution

$\textrm{VaR}_{t} = - (\mu-1.64\sigma_t)$.

If the model is correct, then the loss should not exceed $\textrm{VaR}_{t}$ in more than 5% of the months.

In [8]:
μ = mean(Rme)

λ = 0.95                         #weight on old volatility
σ² = fill(var(Rme),T)            #RiskMetrics approach to estimate variance     
for t = 2:T
  σ²[t] = λ*σ²[t-1] + (1-λ)*(Rme[t-1]-μ)^2    
end

VaR95 = -(μ .- 1.64*sqrt.(σ²))      #VaR at 95% level

println("let's plot it")

let's plot it


In [9]:
xTicksLoc = Dates.value.([Date(1980);Date(1990);Date(2000);Date(2010)])
xTicksLab = ["1980";"1990";"2000";"2010"]   #crude way of getting tick marks right

plot1 = plot(dN,VaR95,color=:blue,legend=false,xticks=(xTicksLoc,xTicksLab))
plot!(ylim=(0,11))
title!("1-month Value at Risk (95%) for US equity market")
ylabel!("%")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1980 
 
 
 1990 
 
 
 2000 
 
 
 2010 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 1-month Value at Risk (95%) for US equity market 
 
 
 % 
 
<polyline clip-path="url(#clip02)" style="stroke:#0000ff; stroke-width:1; stroke-opacity:1; fill:none" points="
 54.5598,158.308 55.9436,160.66 57.1935,161.982 58.5773,160.3 59.9164,166.146 61.3002,169.655 62.6394,172.147 64.0232,177.776 65.407,175.317 66.7461,180.357 
 68.1299,162.929 69.4691,162.259 70.8529,167.676 72.2367,165.637 73.5312,170.865 74.915,124.865 76.2542,128.869 77.638,130.277 78.9772,135.303 80.361,133.226 
 81.7448,139.522 83.0839,145.307 84.4677,151.563 85.8069,136.458 87.1907,135.606 88.5745,133.964 89.8244,140.561 91.2082,144.906 92.5473,149.099 93.9311,155.274 
 95.2703,158.855 96.6541,163.502 98.0379,153.418 99.3771,141.292 100.761,143.027 102.1,147.137 103.484,148.449 104.868,150.282 106.118,144.6 107.501,149.123 
 108.84,153.488 110.224,154.031 111.563,155.777 112.947,157.979 114.331,134.18 115.67,140.692 117.054,118.488 118.393,121.673 119.777,128.577 121.161,133.189 
 122.411,138.975 123.794,144.128 125.134,140.605 126.517,147.038 127.857,151.596 129.24,152.18 130.624,158.034 131.963,164.008 133.347,164.828 134.686,169.84 
 136.07,173.833 137.454,177.267 138.748,174.456 140.132,180.031 141.471,185.041 142.855,176.707 144.194,181.922 145.578,183.492 146.962,159.892 148.301,165.255 
 149.685,170.3 151.024,174.249 152.408,179.44 153.792,168.694 155.042,174.338 156.425,179.323 157.765,184.031 159.148,183.465 160.488,188.712 161.871,193.413 
 163.255,197.616 164.594,193.605 165.978,195.308 167.317,189.605 168.701,191.737 170.085,196.864 171.335,189.479 172.719,188.977 174.058,192.987 175.442,192.833 
 176.781,197.913 178.164,186.331 179.548,181.818 180.887,163.152 182.271,164.687 183.61,170.431 184.994,171.949 186.378,138.623 187.628,141.405 189.012,147.368 
 190.351,151.583 191.735,157.676 193.074,160.613 194.458,163.305 195.841,167.139 197.181,170.004 198.564,45.0606 199.904,41.2062 201.287,43.2525 202.671,49.6689 
 203.966,55.1136 205.349,62.2608 206.689,70.6769 208.072,78.6406 209.412,83.1397 210.795,90.2921 212.179,94.4807 213.518,100.679 214.902,108.054 216.241,113.452 
 217.625,120.387 219.009,120.088 220.259,125.045 221.643,131.679 222.982,135.122 224.366,140.021 225.705,145.608 227.089,141.014 228.472,147.235 229.811,152.964 
 231.195,154.173 232.534,160.195 233.918,166.022 235.302,152.828 236.552,158.923 237.936,164.501 239.275,165.393 240.659,154.739 241.998,160.046 243.382,164.569 
 244.765,140.319 246.105,135.501 247.488,140.397 248.828,139.205 250.211,144.866 251.595,147.338 252.845,142.324 254.229,147.8 255.568,153.874 256.952,157.461 
 258.291,155.236 259.675,157.691 261.059,162.946 262.398,167.453 263.782,173.035 265.121,171.95 266.505,150.708 267.888,156.572 269.183,162.541 270.567,165.246 
 271.906,171.004 273.29,176.649 274.629,179.662 276.013,182.148 277.396,184.794 278.736,190.061 280.119,195.218 281.459,196.853 282.842,201.586 284.226,206.41 
 285.476,211.148 286.86,214.785 288.199,215.093 289.583,217.927 290.922,222.364 292.306,226.394 293.69,226.843 295.029,230.833 296.413,234.579 297.752,235.777 
 299.135,239.269 300.519,240.923 301.769,240.292 303.153,231.704 304.492,235.827 305.876,239.849 307.215,237.899 308.599,239.836 309.983,239.107 311.322,239.648 
 312.706,243.477 314.045,237.771 315.429,241.723 316.812,245.138 318.062,245.02 319.446,247.602 320.785,250.351 322.169,251.636 323.508,253.289 324.892,252.552 
 326.276,256.14 327.615,256.368 328.999,257.52 330.338,255.89 331.722,259.314 333.106,261.171 334.4,264.345 335.784,267.639 337.123,269.682 338.507,271.369 
 339.846,272.664 341.23,254.696 342.614,255.841 343.953,250.798 345.337,254.384 346.676,243.83 348.06,245.492 349.443,241.125 350.693,244.479 352.077,235.147 
 353.416,234.884 354.8,224.061 356.139,223.761 357.523,211.582 358.907,208.432 360.246,20

# Options

## Black-Scholes Option Price

Let $S$ be the the current spot price of an asset and $y$ be the interest rate.

The Black-Scholes formula for a European call option (on that asset) with strike price $K$ and time to expiration $m$ is

$C  =S\Phi\left(  d_{1}\right)  -e^{-ym}K\Phi\left(d_{2}\right)$, where

$d_{1} =\frac{\ln(S/K)+(y+\sigma^{2}/2)m}{\sigma\sqrt{m}} \ \text{ and } \ d_{2}=d_{1}-\sigma\sqrt{m}$ 

and where $\Phi(d)$ denotes the probability of $x\leq d$ when $x$ has an $N(0,1)$ distribution. All variables except the volatility ($\sigma$) are directly observable. 

In [10]:
function Φ(x)
    #Calculates Pr(z<=x) for N(0,1) variable z
    Pr = cdf(Normal(0,1),x)         #Distributions package
  return Pr
end

function OptionBlackSPs(S,K,m,y,σ)
#Calculates Black-Scholes european call option price
  d1 = ( log(S/K) + (y+1/2*σ^2)*m ) / (σ*sqrt(m))
  d2 = d1 - σ*sqrt(m)
  c  = S*Φ(d1) - exp(-y*m)*K*Φ(d2)
  return c
end

OptionBlackSPs (generic function with 1 method)

In [11]:
σ = 0.4
c1 = OptionBlackSPs(10,10,0.5,0.1,σ)
printlnPs("\n","call price according to Black-Scholes: ",c1)

K = linspace(7,13,51)
c = OptionBlackSPs.(10,K,0.5,0.1,σ)

println("let's plot it")


call price according to Black-Scholes:      1.358
let's plot it


In [12]:
plot1 = plot(K,c,color=:red,legend=false)
title!("Black-Scholes call option price (for different strike prices)")
xlabel!("strike price")
ylabel!("option price")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 7 
 
 
 8 
 
 
 9 
 
 
 10 
 
 
 11 
 
 
 12 
 
 
 13 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 2.5 
 
 
 3.0 
 
 
 Black-Scholes call option price (for different strike prices) 
 
 
 strike price 
 
 
 option price 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:1; stroke-opacity:1; fill:none" points="
 47.9701,31.4961 58.617,42.6796 69.2639,53.7251 79.9108,64.6249 90.5577,75.3715 101.205,85.9577 111.852,96.3764 122.498,106.621 133.145,116.686 143.792,126.565 
 154.439,136.252 165.086,145.744 175.733,155.035 186.38,164.122 197.027,173.001 207.674,181.67 218.32,190.126 228.967,198.367 239.614,206.392 250.261,214.2 
 260.908,221.791 271.555,229.164 282.202,236.321 292.849,243.262 303.496,249.989 314.143,256.502 324.789,262.805 335.436,268.899 346.083,274.788 356.73,280.474 
 367.377,285.96 378.024,291.25 388.671,296.348 399.318,301.257 409.965,305.981 420.612,310.525 431.258,314.893 441.905,319.089 452.552,323.117 463.199,326.983 
 473.846,330.691 484.493,334.244 495.14,337.649 505.787,340.909 516.434,344.029 527.08,347.014 537.727,349.868 548.374,352.596 559.021,355.202 569.668,357.69 
 580.315,360.065 
 "/>

# Implied Volatility

is the $\sigma$ value that makes the Black-Scholes equation give the same option price as observed on the market. It is often interpreted as the "market uncertainty."

The next cell uses the call option price calculated above as being the "market price." The implied volatility should then be the same as what we used above.

In [13]:
                                #simple (crude) way to solve for implied vol
iv = fzero(σ->OptionBlackSPs(10,10,0.5,0.1,σ)-c1,0.00001,5)

printlnPs("Implied volatility: ",iv,", compare with: $σ")

Implied volatility:      0.400, compare with: 0.4


The next few cells instead some data on options on German government bonds. 

In [14]:
#  LIFFE Bunds option data, trade date April 6, 1994
K = [                        #strike prices; Mx1 vector
      92.00;  94.00;  94.50;  95.00;  95.50;  96.00;  96.50;  97.00;
      97.50;  98.00;  98.50;  99.00;  99.50;  100.0;  100.5;  101.0;
     101.5;  102.0;  102.5;  103.0;  103.5 ];
C = [                        #call prices; Mx1 vector
      5.13;    3.25;    2.83;    2.40;    2.00;    1.64;    1.31;    1.02;
      0.770;   0.570;   0.400;   0.280;   0.190;   0.130;  0.0800;  0.0500;
      0.0400;  0.0300;  0.0200;  0.0100;  0.0100 ];
S = 97.05                #spot price
m = 48/365               #time to expiry in years
y = 0.0                  #Interest rate: LIFFE=>no discounting
N = length(K)

21

In [15]:
iv = fill(NaN,N)
for i = 1:N
  iv[i] = fzero(sigma->OptionBlackSPs(S,K[i],m,y,sigma)-C[i],0.00001,5)
end

println("Strike and iv for data: ")
printmat([K iv])

println("let's plot it")

Strike and iv for data: 
    92.000     0.094
    94.000     0.081
    94.500     0.081
    95.000     0.078
    95.500     0.075
    96.000     0.074
    96.500     0.072
    97.000     0.071
    97.500     0.070
    98.000     0.069
    98.500     0.068
    99.000     0.067
    99.500     0.067
   100.000     0.068
   100.500     0.067
   101.000     0.067
   101.500     0.070
   102.000     0.073
   102.500     0.074
   103.000     0.072
   103.500     0.077

let's plot it


In [16]:
plot1 = plot(K,iv,color=:red,legend=false)
title!("Implied volatility, Bunds options April 6, 1994")
xlabel!("strike price")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 92 
 
 
 94 
 
 
 96 
 
 
 98 
 
 
 100 
 
 
 102 
 
 
 0.070 
 
 
 0.075 
 
 
 0.080 
 
 
 0.085 
 
 
 0.090 
 
 
 Implied volatility, Bunds options April 6, 1994 
 
 
 strike price

# Mean-variance Frontier

Given a vector of average returns ($\mu$) and a variance-covariance matrix ($\Sigma$), the mean-variance frontier show the lowest possible portfolio uncertainty for a given expected portfolio return. It is thus the solution to a quadratic minimization problem. The cells below will use the explicit (matrix) formulas for this solution, but we often have to resort to numerical methods when there are portfolio restrictions.

It is typically plotted with the portfolio standard deviation on the horizontal axis and the portfolio expected return on the vertical axis.

In [17]:
μ = [0.115; 0.095; 0.06]        #expected returns
Σ  = [166  34  58;              #covariance matrix
       34  64   4;
       58   4 100]/100^2                  
Rf = 0.03                       #riskfree return (an interest rate) 

println("μ, Σ and Rf: ")
printmat(μ)
printmat(Σ)
printmat(Rf)

μ, Σ and Rf: 
     0.115
     0.095
     0.060

     0.017     0.003     0.006
     0.003     0.006     0.000
     0.006     0.000     0.010

     0.030



The functions in the next cell use the explicit solution to the minimization problem. They assume that there are no portfolio restrictions. 

In [18]:
function MVCalc(mustar,μ,Σ)           #calculate the std of a portfolio on MVF of risky assets
    n    = length(μ)                  #mustar is a scalar, μ a vector and Σ a matrix  
    oneV = ones(n) 
    Σ_1  = inv(Σ)
    A    = μ'Σ_1*μ                  
    B    = μ'Σ_1*oneV
    C    = oneV'Σ_1*oneV
    λ    = (C*mustar - B)/(A*C-B^2)
    δ    = (A-B*mustar)/(A*C-B^2)
    w    = Σ_1 *(μ*λ + δ*oneV)
    StdRp = sqrt(w'Σ*w)
    return StdRp,w
end

function MVCalcRf(mustar,μ,Σ,Rf)           #calculates the std of a portfolio on MVF of (Risky,Riskfree)
    n     = length(μ)
    μe    = μ .- Rf                        #expected excess returns            
    Σ_1   = inv(Σ)
    w     = (mustar - Rf)/(μe'Σ_1*μe) * Σ_1 *μe
    StdRp = sqrt(w'Σ*w)
    return StdRp,w
end

MVCalcRf (generic function with 1 method)

In [19]:
mustar = linspace(Rf,0.15,101)
L      = length(mustar)
StdRp  = fill(NaN,L)
for i = 1:L
    StdRp[i] = MVCalc(mustar[i],μ,Σ)[1]    #[1] to get first output
end    

In [20]:
plot3 = plot(StdRp*100,mustar*100,legend=nothing,color=:red,xlim=(0,15),ylim=(0,15))
scatter!(sqrt.(diag(Σ))*100,μ*100)
title!("MVF, only risky assets")
xlabel!("Std(Rp), %")
ylabel!("ERp, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 MVF, only risky assets 
 
 
 Std(Rp), % 
 
 
 ERp, % 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:1; stroke-opacity:1; fill:none" points="
 503.299,294.352 495.169,291.723 487.091,289.094 479.065,286.466 471.096,283.837 463.186,281.209 455.339,278.58 447.558,275.952 439.848,273.323 432.213,270.695 
 424.656,268.066 417.184,265.437 409.801,262.809 402.513,260.18 395.325,257.552 388.244,254.923 381.277,252.295 374.43,249.666 367.712,247.038 361.131,244.409 
 354.696,241.78 348.414,239.152 342.298,236.523 336.356,233.895 330.6,231.266 325.041,228.638 319.691,226.009 314.562,223.381 309.668,220.752 305.022,218.123 
 300.636,215.495 296.526,212.866 292.704,210.238 289.183,207.609 285.978,204.981 283.101,202.352 280.563,199.724 278.376,197.095 276.551,194.466 275.094,191.838 
 274.014,189.209 273.316,186.581 273.003,183.952 273.077,181.324 273.537,178.695 274.382,176.067 275.607,173.438 277.205,170.809 279.17,168.181 281.492,165.552 
 284.161,162.924 287.165,160.295 290.492,157.667 294.129,155.038 298.063,152.41 302.28,149.781 306.767,147.152 311.51,144.524 316.495,141.895 321.709,139.267 
 327.141,136.638 332.776,134.01 338.605,131.381 344.615,128.753 350.795,126.124 357.137,123.495 363.629,120.867 370.264,118.238 377.032,115.61 383.925,112.981 
 390.937,110.353 398.059,107.724 405.286,105.096 412.611,102.467 420.029,99.8385 427.534,97.2099 435.121,94.5814 442.785,91.9528 450.523,89.3243 458.329,86.6957 
 466.201,84.0672 474.134,81.4386 482.125,78.8101 490.171,76.1815 498.27,73.5529 506.417,70.9244 514.612,68.2958 522.851,65.6673 531.132,63.0387 539.453,60.4102 
 547.813,57.7816 556.208,55.1531 564.638,52.5245 573.1,49.8959 581.594,47.2674 590.118,44.6388 598.67,42.0103 607.249,39.3817 615.854,36.7532 624.484,34.1246 
 633.137,31.4961 
 "/>

In [21]:
StdRpRf  = fill(NaN,L)
for i = 1:L
    StdRpRf[i] = MVCalcRf(mustar[i],μ,Σ,Rf)[1]
end    

In [22]:
plot4 = plot(StdRp*100,mustar*100,legend=nothing,color=:red,xlim=(0,15),ylim=(0,15))
plot!(StdRpRf*100,mustar*100,legend=nothing,color=:blue)
scatter!(sqrt.(diag(Σ))*100,μ*100)
title!("MVF, risky and riskfree assets")
xlabel!("Std(Rp), %")
ylabel!("ERp, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 MVF, risky and riskfree assets 
 
 
 Std(Rp), % 
 
 
 ERp, % 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:1; stroke-opacity:1; fill:none" points="
 503.299,294.352 495.169,291.723 487.091,289.094 479.065,286.466 471.096,283.837 463.186,281.209 455.339,278.58 447.558,275.952 439.848,273.323 432.213,270.695 
 424.656,268.066 417.184,265.437 409.801,262.809 402.513,260.18 395.325,257.552 388.244,254.923 381.277,252.295 374.43,249.666 367.712,247.038 361.131,244.409 
 354.696,241.78 348.414,239.152 342.298,236.523 336.356,233.895 330.6,231.266 325.041,228.638 319.691,226.009 314.562,223.381 309.668,220.752 305.022,218.123 
 300.636,215.495 296.526,212.866 292.704,210.238 289.183,207.609 285.978,204.981 283.101,202.352 280.563,199.724 278.376,197.095 276.551,194.466 275.094,191.838 
 274.014,189.209 273.316,186.581 273.003,183.952 273.077,181.324 273.537,178.695 274.382,176.067 275.607,173.438 277.205,170.809 279.17,168.181 281.492,165.552 
 284.161,162.924 287.165,160.295 290.492,157.667 294.129,155.038 298.063,152.41 302.28,149.781 306.767,147.152 311.51,144.524 316.495,141.895 321.709,139.267 
 327.141,136.638 332.776,134.01 338.605,131.381 344.615,128.753 350.795,126.124 357.137,123.495 363.629,120.867 370.264,118.238 377.032,115.61 383.925,112.981 
 390.937,110.353 398.059,107.724 405.286,105.096 412.611,102.467 420.029,99.8385 427.534,97.2099 435.121,94.5814 442.785,91.9528 450.523,89.3243 458.329,86.6957 
 466.201,84.0672 474.134,81.4386 482.125,78.8101 490.171,76.1815 498.27,73.5529 506.417,70.9244 514.612,68.2958 522.851,65.6673 531.132,63.0387 539.453,60.4102 
 547.813,57.7816 556.208,55.1531 564.638,52.5245 573.1,49.8959 581.594,47.2674 590.118,44.6388 598.67,42.0103 607.249,39.3817 615.854,36.7532 624.484,34.1246 
 633.137,31.4961 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#0000ff; stroke-width:1; stroke-opacity:1; fill:none" points="
 44.6753,294.352 49.3554,291.723 54.0355,289.094 58.7155,286.466 63.3956,283.837 68.0757,281.209 72.7557,278.58 77.4358,275.952 82.1159,273.323 86.7959,270.695 
 91.476,268.066 96.1561,265.437 100.836,262.809 105.516,260.18 110.196,257.552 114.876,254.923 119.556,252.295 124.237,249.666 128.917,247.038 133.597,244.409 
 138.277,241.78 142.957,239.152 147.637,236.523 152.317,233.895 156.997,231.266 161.677,228.638 166.357,226.009 171.037,223.381 175.717,220.752 180.397,218.123 
 185.077,215.495 189.757,212.866 194.438,210.238 199.118,207.609 203.798,204.981 208.478,202.352 213.158,199.724 217.838,197.095 222.518,194.466 227.198,191.838 
 231.878,189.209 236.558,186.581 241.238,183.952 245.918,181.324 250.598,178.695 255.278,176.067 259.959,173.438 264.639,170.809 269.319,168.181 273.999,165.552 
 278.679,162.924 283.359,160.295 288.039,157.667 292.719,155.038 297.399,152.41 302.079,149.781 306.759,147.152 311.439,144.524 316.119,141.895 320.799,139.267 
 325.479,136.638 330.16,134.01 334.84,131.381 339.52,128.753 344.2,126.124 348.88,123.495 353.56,120.867 358.24,118.238 362.92,115.61 367.6,112.981 
 372.28,110.353 376.96,107.724 381.64,105.096 386.32,102.467 391,99.8385 395.681,97.2099 400.361,94.5814 405.041,91.9528 409.721,89.3243 414.401,86.6957 
 419.081,84.0672 423.761,81.4386 428.441,78.8101 433.121,76.1815 437.801,73.5529 442.481,70.9244 447.161,68.2958 451.841,65.6673 456.521,63.0387 461.201,60.4102 
 465.882,57.7816 470.562,55.1531 475.242,52.5245 479.922,49.8959 484.602,47.2674 489.282,44.6388 493.962,42.0103 498.642,39.3817 503.322,36.7532 508.002,34.1246 
 512.682,31.4961 
 "/>

# (extra) Mean-variance Frontier without Short Selling

The code below solves (numerically) the following minimization problem 

$\min \text{Var}(R_p) \> \text{ s.t. } \>  \text{E}R_p = \mu^*$,
 
and where we require $w_i\ge 0$
 
The code below calls on
```
quadprog(zeros(N),Sigma,A,'=',b,zeros(N),ones(N),IpoptSolver(print_level=1))
```

This means that we minimize
    $0'w + w' \Sigma w/2$ subject to the restrictions $A w = b$ and $0 \leq w \leq 1$. The equality constraints ($A w = b$) are used to impose $1'w=1$ and $\mu'w = \mu^*$ (se the code below).
    
The packages MathProgBase and Ipopt are documented at https://github.com/JuliaOpt/MathProgBase.jl and https://github.com/JuliaOpt/Ipopt.jl.

In [23]:
using MathProgBase, Ipopt

In [24]:
function MeanVarNoSSPs(mu,Sigma,mustar)   #MV with no short-sales, numerical minimization
    (w_p,StdRp) = (NaN,NaN)     #default result (if no feasible solution)        
    N = length(mu)
    if minimum(mu) <= mustar <= maximum(mu)  #try only if feasible
        A   = [ones(1,N);mu']                #1'w=1, mu'w = mustar
        b   = [1;mustar]
        Sol = quadprog(zeros(N),Sigma,A,'=',b,zeros(N),ones(N),IpoptSolver(print_level=1))
        if Sol.status == :Optimal
            w_p = Sol.sol
            StdRp = sqrt(w_p'Sigma*w_p)
        end    
    end    
    return StdRp,w_p
end

MeanVarNoSSPs (generic function with 1 method)

In [25]:
Std_no_ss = fill(NaN,length(mustar))
for i = 1:length(mustar)
    Std_no_ss[i] = MeanVarNoSSPs(μ,Σ,mustar[i])[1]    
end


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************



In [26]:
plot(StdRp*100,mustar*100,color=:red,linewidth=2,label="Risky only")
plot!(Std_no_ss*100,mustar*100,color=:green,line=(:dash,2),label="Risky only (no ss) ")
plot!(xlim=(0,15),ylim=(0,15))
title!("MVF (portfolio restrictions)")
xlabel!("Std(Rp), %")
ylabel!("ERp, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 MVF (portfolio restrictions) 
 
 
 Std(Rp), % 
 
 
 ERp, % 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:2; stroke-opacity:1; fill:none" points="
 503.299,294.352 495.169,291.723 487.091,289.094 479.065,286.466 471.096,283.837 463.186,281.209 455.339,278.58 447.558,275.952 439.848,273.323 432.213,270.695 
 424.656,268.066 417.184,265.437 409.801,262.809 402.513,260.18 395.325,257.552 388.244,254.923 381.277,252.295 374.43,249.666 367.712,247.038 361.131,244.409 
 354.696,241.78 348.414,239.152 342.298,236.523 336.356,233.895 330.6,231.266 325.041,228.638 319.691,226.009 314.562,223.381 309.668,220.752 305.022,218.123 
 300.636,215.495 296.526,212.866 292.704,210.238 289.183,207.609 285.978,204.981 283.101,202.352 280.563,199.724 278.376,197.095 276.551,194.466 275.094,191.838 
 274.014,189.209 273.316,186.581 273.003,183.952 273.077,181.324 273.537,178.695 274.382,176.067 275.607,173.438 277.205,170.809 279.17,168.181 281.492,165.552 
 284.161,162.924 287.165,160.295 290.492,157.667 294.129,155.038 298.063,152.41 302.28,149.781 306.767,147.152 311.51,144.524 316.495,141.895 321.709,139.267 
 327.141,136.638 332.776,134.01 338.605,131.381 344.615,128.753 350.795,126.124 357.137,123.495 363.629,120.867 370.264,118.238 377.032,115.61 383.925,112.981 
 390.937,110.353 398.059,107.724 405.286,105.096 412.611,102.467 420.029,99.8385 427.534,97.2099 435.121,94.5814 442.785,91.9528 450.523,89.3243 458.329,86.6957 
 466.201,84.0672 474.134,81.4386 482.125,78.8101 490.171,76.1815 498.27,73.5529 506.417,70.9244 514.612,68.2958 522.851,65.6673 531.132,63.0387 539.453,60.4102 
 547.813,57.7816 556.208,55.1531 564.638,52.5245 573.1,49.8959 581.594,47.2674 590.118,44.6388 598.67,42.0103 607.249,39.3817 615.854,36.7532 624.484,34.1246 
 633.137,31.4961 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#008000; stroke-width:2; stroke-opacity:1; fill:none" stroke-dasharray="8, 5" points="
 401.768,228.638 390.154,226.009 378.835,223.381 367.843,220.752 357.214,218.123 346.984,215.495 337.197,212.866 327.897,210.238 319.134,207.609 310.963,204.981 
 303.437,202.352 296.616,199.724 290.558,197.095 285.32,194.466 280.958,191.838 277.52,189.209 275.047,186.581 273.572,183.952 273.113,181.324 273.537,178.695 
 274.382,176.067 275.607,173.438 277.205,170.809 279.17,168.181 281.492,165.552 284.161,162.924 287.165,160.295 290.492,157.667 294.129,155.038 298.063,152.41 
 302.28,149.781 306.767,147.152 311.51,144.524 316.495,141.895 321.709,139.267 327.761,136.638 336.165,134.01 346.806,131.381 359.456,128.753 373.884,126.124 
 389.867,123.495 407.198,120.867 425.695,118.238 445.196,115.61 465.561,112.981 486.671,110.353 
 "/>
 
 
 
 
 Risky only 
 
 
 
 Risky only (no ss)